# 🤖 AI, CA3, Machine Learning 📚  

* **Name** : Mohammad Sadra 🖊  
* **Last Name** :  Abbasi 📝  
* **SID** : 810101469 🆔

# IMPORT


In [ ]:
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Preprocess

In [ ]:
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation and special characters (keep only a-z and spaces)
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokenize (split by space)
    words = text.split()
    # Remove stopwords & Lemmatize
    clean_words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return " ".join(clean_words)


# Build Vocab and vectorisation

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

class Vectorizer:
    def __init__(self):
        self.vocab = None
        self.count_vectorizer = None

    def build_vocab(self, cleaned_set):
        self.count_vectorizer = CountVectorizer(min_df=1)
        
        self.count_vectorizer.fit(cleaned_set)
        
        self.vocab = self.count_vectorizer.vocabulary_

        return self.vocab
    
    def create_vectors(self, cleaned_set):
        self.vocab = self.build_vocab(cleaned_set)
        
        if self.vocab is None:
            raise ValueError("Vocabulary is empty.")
            
        X_counts = self.count_vectorizer.transform(cleaned_set)
        
        binary_gen = CountVectorizer(vocabulary=self.vocab, binary=True)
        X_binary = binary_gen.transform(cleaned_set)
        
        return X_counts, X_binary
    

# Modeling and train

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

class TrainModel:
    def __init__(self):
        self.models = {}

    def train_models(self, X_train_counts, X_train_binary, y_train):
        mnb = MultinomialNB()
        mnb.fit(X_train_counts, y_train)
        self.models['MultinomialNB'] = (mnb, 'counts')

        bnb = BernoulliNB()
        bnb.fit(X_train_binary, y_train)
        self.models['BernoulliNB'] = (bnb, 'binary')

        dt = DecisionTreeClassifier(max_depth=4, random_state=42)
        dt.fit(X_train_counts, y_train)
        self.models['DecisionTree'] = (dt, 'counts')

    def evaluate_models(self, X_val_counts, X_val_binary, y_val):
        results_list = []

        for name, (model, data_type) in self.models.items():
            if data_type == 'counts':
                X_test = X_val_counts
            elif data_type == 'binary':
                X_test = X_val_binary
            
            y_pred = model.predict(X_test)

            metrics = {
                "Model": name,
                "Accuracy": accuracy_score(y_val, y_pred),
                "Precision": precision_score(y_val, y_pred, zero_division=0),
                "Recall": recall_score(y_val, y_pred, zero_division=0),
                "F1 Score": f1_score(y_val, y_pred, zero_division=0)
            }
            results_list.append(metrics)

            cm = confusion_matrix(y_val, y_pred)
            disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Spam', 'Important'])
            
            fig, ax = plt.subplots(figsize=(4, 4))
            disp.plot(cmap=plt.cm.Blues, ax=ax, colorbar=False)
            plt.title(f"Confusion Matrix\n{name}")
            plt.show()
            print("-" * 60)

        return pd.DataFrame(results_list)
    
    def generate_best_submission(self, X_test, message_texts, output_folder, model_key='MultinomialNB' , name = "submission.csv"):
        if model_key not in self.models:
            raise ValueError(f"Model '{model_key}' not found!")
        
        model, _ = self.models[model_key]
        
        predictions = model.predict(X_test)
        
        submission = pd.DataFrame({
            'message_body': message_texts.values,
            'prediction': predictions
        })
        
        file_path = os.path.join(output_folder, name)
        submission.to_csv(file_path, index=False)
        
        print(f"File saved to: {file_path}")        

# Evaluation

In [ ]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('./part2_dataset/part2_dataset.csv')

df['clean_text'] = df['message_body'].apply(clean_text)

vectorizer = Vectorizer()
X_counts, X_binary = vectorizer.create_vectors(df['clean_text'])
y = df['is_important']

X_train_c, X_val_c, y_train, y_val = train_test_split(
    X_counts, y, test_size=0.3, random_state=42
)

X_train_b, X_val_b, _, _ = train_test_split(
    X_binary, y, test_size=0.3, random_state=42
)

trainer = TrainModel()

trainer.train_models(X_train_c, X_train_b, y_train)

final_report = trainer.evaluate_models(X_val_c, X_val_b, y_val)

display(final_report)

##


<div dir="rtl" style="text-align: justify; line-height: 1.8;">
    <h3 style="margin-top: 15px;">بررسی و تحلیل خروجی</h3>
</div>

<table style="width: 100%; text-align: center; margin: 25px 0;">
    <tr>
        <td><img src="./Assets/BernoulliNB.png" width="800"></td>
        <td><img src="./Assets/MultinomialNB_res.png" width="800"></td>
        <td><img src="./Assets/DecisionTree.png" width="800"></td>
        <td><img src="./Assets/part2_final_res.png" width="800"></td>
    </tr>
    <tr>
        <td>Naive bayes with binary vector</td>
        <td>Naive bayes with counting vector</td>
        <td>Destition tree with depth = 4</td>
        <td>final result</td>‍
    </tr>
</table>

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
مدل Bernoulli Naive Bayes توانسته است تمامی ایمیل های مهم را به درستی تشخیص دهد و از این نظر Recall کاملی داشته باشد، اما در مقابل، نرخ نسبتا بالایی از ایمیل های اسپم را نیز به اشتباه در دسته ایمیل های مهم قرار داده است. دلیل اصلی این رفتار آن است که این مدل صرفا بر اساس وجود یا عدم وجود واژه ها در متن ایمیل تصمیم گیری می کند و هیچ توجهی به تعداد تکرار آن ها ندارد. در نتیجه، وجود چند واژه کلیدی آموزشی در یک ایمیل اسپم می تواند باعث شود مدل آن ایمیل را به عنوان ایمیل مهم تشخیص دهد، حتی اگر این واژه ها تنها به صورت سطحی در متن ظاهر شده باشند.
</p>

<p>
در مورد مدل درخت تصمیم نیز با توجه به ماتریس درهم ریختگی و متریک های به دست آمده می توان مشاهده کرد که رفتار آن تا حد زیادی مشابه مدل Bernoulli Naive Bayes است. این مدل نیز بیشتر بر اساس وجود برخی واژه های خاص اقدام به تفکیک ایمیل ها کرده و به همین دلیل دچار بایاس نسبت به این واژه ها شده است. اگرچه درخت تصمیم از نظر تفسیرپذیری مزیت دارد و می تواند قوانینی ساده از جنس وجود یا عدم وجود برخی کلمات را یاد بگیرد، اما در این مسئله خاص توان تعمیم مناسبی برای تشخیص ایمیل های اسپم نداشته و در نتیجه عملکرد کلی ضعیف تری نسبت به سایر مدل ها ارائه داده است.
</p>

<p>
در مقابل، مدل Multinomial Naive Bayes بهترین عملکرد را در میان سه مدل مورد بررسی نشان داده است. این مدل علاوه بر تشخیص صحیح تمامی ایمیل های مهم، توانسته است تعداد بسیار کمتری از ایمیل های اسپم را به اشتباه مهم تشخیص دهد. علت اصلی این بهبود عملکرد، در نظر گرفتن فرکانس وقوع هر واژه در متن ایمیل است. در این نمایش، ایمیل های واقعا آموزشی که معمولا شامل تکرار چندین باره واژه های مرتبط با درس، تمرین و امتحان هستند، به خوبی از ایمیل های اسپم که ممکن است تنها یک یا دو بار از این واژه ها استفاده کنند، تفکیک می شوند.
</p>

<p>
به طور کلی نتایج نشان می دهد که در این مسئله، نوع نمایش ویژگی ها تاثیر بسیار بیشتری نسبت به پیچیدگی مدل داشته است. استفاده از یک مدل ساده مانند Naive Bayes، در کنار Feature Engineering مناسب، منجر به عملکردی بهتر از مدلی پیچیده تر مانند درخت تصمیم شده و اهمیت انتخاب صحیح نمایش داده ها در مسائل پردازش متن را به خوبی نشان می دهد.
</p>

</div>



# Runner

In [ ]:
output_folder = "part2_submissions"
test_file = "part2_test.csv"

df_test = pd.read_csv(test_file)
df_test['clean_text'] = df_test['message_body'].apply(clean_text)

X_test_counts = vectorizer.count_vectorizer.transform(df_test['clean_text'])
binary_gen = CountVectorizer(vocabulary=vectorizer.vocab, binary=True)
X_test_binary = binary_gen.transform(df_test['clean_text'])

trainer.generate_best_submission(
    X_test=X_test_counts, 
    message_texts=df_test['message_body'], 
    output_folder=output_folder,
    model_key='MultinomialNB',
    name = "MultinomialNB_results.csv"
)

trainer.generate_best_submission(
    X_test=X_test_binary, 
    message_texts=df_test['message_body'], 
    output_folder=output_folder,
    model_key='BernoulliNB',
    name = "BernoulliNB_results.csv"
)

trainer.generate_best_submission(
    X_test=X_test_counts, 
    message_texts=df_test['message_body'], 
    output_folder=output_folder,
    model_key='DecisionTree',
    name = "DecisionTree_results.csv"
)

# سوالات

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
در نمایش bag-of-words، هر ایمیل به مجموعه‌ای از واژه‌ها در نظر گرفته شده و ترتیب و ساختار جمله‌ها در نظر گرفته نمی‌شود. درنتیجه بسیاری از وابستگی‌هایی که در متن طبیعی وجود دارند، حذف شده و واژه‌ها مستقل از محتوا خواهند بود.اگرچه این فرض استقلال به طور کامل برقرار نیست، اما برای مسئله‌ای مانند تشخیص ایمیل مهم از اسپم، این ساده‌سازی تا حد قابل قبولی منطقی به نظر می‌رسد. در این آزمایش، مدل بیشتر به حضور یا فراوانی واژه‌های مرتبط با فضای آموزشی واکنش نشان می‌دهد و همین موضوع باعث شده Naive Bayes بدون نیاز به مدل‌سازی روابط پیچیده، عملکرد مناسبی داشته باشد.
</p>

<p>واژه‌هایی مانند exam، deadline و homework معمولا به صورت هم‌زمان در ایمیل‌های آموزشی دیده می‌شوند و مستقل از یکدیگر نیستند که فرض استقلال ویژگی هارا نقض می کنند اما در Naive Bayes این نقض لزوما به افت شدید عملکرد منجر نمی‌شود مدل ممکن است تاثیر این واژه‌ها را بیش از مقدار واقعی برآورد کند، اما طبق نتیجه حاصل شده لیبل نهایی در اغلب موارد درست خواهد بود
</p>

<p>
با بررسی خروجی‌ها می‌توان مشاهده کرد که درخت تصمیم توانسته است الگوهای وجود هم‌زمان چند واژه خاص را یاد بگیرد و تصمیم‌های خود را بر اساس آن جهت دهی کند اما برخی ایمیل‌های اسپم را نیز صرفا به دلیل داشتن چند واژه مشترک با ایمیل‌های آموزشی، در دسته ایمیل‌های مهم قرار داده است. در مقایسه، Naive Bayes به‌ویژه در حالت شمارشی، با در نظر گرفتن فراوانی واژه‌ها توانسته است تفکیک دقیق‌تری بین ایمیل‌های مهم و اسپم ایجاد کند و در مجموع عملکرد بهتری ارائه دهد.
</p>

</div>
